In [1]:
import pandas as pd
import numpy as np
import os
import time
from IPython.display import clear_output

In [2]:
root = '/home/MIBS/facenet'
target_folder = os.path.join(root, 'embedding', 'similarity')

## Read data

In [3]:
df_sim = pd.read_csv(os.path.join(target_folder, 'similarity_sample.csv'), index_col=0)
df_sim.head()

,1000046113_190517.jpg,1000046113_190542.jpg,1000046113_190396.jpg,1000046113_190227.jpg,1000046113_190351.jpg,1000046113_190136.jpg,1000046113_190150.jpg,1000046113_190142.jpg,1000046113_190162.jpg,1000046113_190247.jpg,...,9738015_377684.jpg,9738015_8590.jpg,9738015_8606.jpg,9738015_15019.jpg,9738015_15000.jpg,9738015_8424.jpg,9738015_8564.jpg,9738015_377756.jpg,9738015_8391.jpg,9738015_15206.jpg
1000046113_190517.jpg,0.000000,0.143982,0.214107,0.179404,0.064594,0.087816,0.107404,0.095076,0.097124,0.106982,...,0.860575,0.747911,0.727228,0.693272,0.712363,0.638291,0.857498,0.823187,0.698919,0.717771
1000046113_190542.jpg,0.143982,0.000000,0.208397,0.260087,0.198990,0.142480,0.217972,0.214152,0.145049,0.208114,...,0.732235,0.679044,0.674223,0.688527,0.710484,0.613962,0.707826,0.751459,0.673050,0.685658
1000046113_190396.jpg,0.214107,0.208397,0.000000,0.489893,0.286222,0.245889,0.352176,0.344287,0.116870,0.384114,...,0.860128,0.794732,0.818045,0.763146,0.787256,0.661241,0.850341,0.886226,0.760610,0.763944
1000046113_190227.jpg,0.179404,0.260087,0.489893,0.000000,0.186223,0.178787,0.125818,0.164370,0.284938,0.074724,...,0.738296,0.677773,0.701092,0.663584,0.645251,0.639914,0.772008,0.688442,0.657168,0.640283
1000046113_190351.jpg,0.064594,0.198990,0.286222,0.186223,0.000000,0.125612,0.157908,0.159176,0.148402,0.151560,...,0.892191,0.754673,0.736680,0.745465,0.755129,0.654258,0.917065,0.808106,0.722302,0.758688


In [4]:
df_sim

,1000046113_190517.jpg,1000046113_190542.jpg,1000046113_190396.jpg,1000046113_190227.jpg,1000046113_190351.jpg,1000046113_190136.jpg,1000046113_190150.jpg,1000046113_190142.jpg,1000046113_190162.jpg,1000046113_190247.jpg,...,9738015_377684.jpg,9738015_8590.jpg,9738015_8606.jpg,9738015_15019.jpg,9738015_15000.jpg,9738015_8424.jpg,9738015_8564.jpg,9738015_377756.jpg,9738015_8391.jpg,9738015_15206.jpg
1000046113_190517.jpg,0.000000,0.143982,0.214107,0.179404,0.064594,0.087816,0.107404,0.095076,0.097124,0.106982,...,0.860575,0.747911,0.727228,0.693272,0.712363,0.638291,0.857498,0.823187,0.698919,0.717771
1000046113_190542.jpg,0.143982,0.000000,0.208397,0.260087,0.198990,0.142480,0.217972,0.214152,0.145049,0.208114,...,0.732235,0.679044,0.674223,0.688527,0.710484,0.613962,0.707826,0.751459,0.673050,0.685658
1000046113_190396.jpg,0.214107,0.208397,0.000000,0.489893,0.286222,0.245889,0.352176,0.344287,0.116870,0.384114,...,0.860128,0.794732,0.818045,0.763146,0.787256,0.661241,0.850341,0.886226,0.760610,0.763944
1000046113_190227.jpg,0.179404,0.260087,0.489893,0.000000,0.186223,0.178787,0.125818,0.164370,0.284938,0.074724,...,0.738296,0.677773,0.701092,0.663584,0.645251,0.639914,0.772008,0.688442,0.657168,0.640283
1000046113_190351.jpg,0.064594,0.198990,0.286222,0.186223,0.000000,0.125612,0.157908,0.159176,0.148402,0.151560,...,0.892191,0.754673,0.736680,0.745465,0.755129,0.654258,0.917065,0.808106,0.722302,0.758688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9738015_8424.jpg,0.638291,0.613962,0.661241,0.639914,0.654258,0.565774,0.627170,0.617883,0.591943,0.570516,...,0.349841,0.339943,0.178568,0.393416,0.368407,0.000000,0.225626,0.389076,0.216939,0.363033
9738015_8564.jpg,0.857498,0.707826,0.850341,0.772008,0.917065,0.767121,0.829019,0.811188,0.788292,0.787758,...,0.379458,0.421981,0.283137,0.476879,0.462412,0.225626,0.000000,0.429955,0.386696,0.446090
9738015_377756.jpg,0.823187,0.751459,0.886226,0.688442,0.808106,0.770393,0.795371,0.792688,0.748108,0.685805,...,0.166385,0.407123,0.291589,0.203697,0.200067,0.389076,0.429955,0.000000,0.301958,0.191935
9738015_8391.jpg,0.698919,0.673050,0.760610,0.657168,0.722302,0.562059,0.612923,0.629705,0.633727,0.608942,...,0.332124,0.123822,0.168079,0.270137,0.186813,0.216939,0.386696,0.301958,-0.000000,0.208213


## Check similarity values

In [ ]:
similarity_data = []
threshold = .25

tp, tn, fp, fn = 0, 0, 0, 0

for column, data_column in df_sim.items():
    idx_1 = column.split('.')[0].split('_')[0]
    tmp_dict = {column: {'same': {'min': None, 'max': None}, 'different': {'min': None, 'max': None}}}
    for row, value in data_column.items():
        idx_2 = row.split('.')[0].split('_')[0]
        if idx_1 == idx_2:
            # same person
            if value < threshold:
                tp += 1
            else:
                fn += 1
            current_min = tmp_dict[column]['same']['min']
            current_max = tmp_dict[column]['same']['max']
            if current_min == None or current_max == None:
                tmp_dict[column]['same']['min'] = value
                tmp_dict[column]['same']['max'] = value
            else:
                if value < tmp_dict[column]['same']['min']:
                    tmp_dict[column]['same']['min'] = value
                if value > tmp_dict[column]['same']['max']:
                    tmp_dict[column]['same']['max'] = value
        else:
            # different person
            if value < threshold:
                fp += 1
            else:
                tn += 1
            current_min = tmp_dict[column]['different']['min']
            current_max = tmp_dict[column]['different']['max']
            if current_min == None or current_max == None:
                tmp_dict[column]['different']['min'] = value
                tmp_dict[column]['different']['max'] = value
            else:
                if value < tmp_dict[column]['different']['min']:
                    tmp_dict[column]['different']['min'] = value
                if value > tmp_dict[column]['different']['max']:
                    tmp_dict[column]['different']['max'] = value
    similarity_data.append(tmp_dict)

In [ ]:
print('TP: {}, TN: {}, FP: {}, FN: {}'.format(tp, tn, fp, fn))

In [ ]:
precision = float(tp) / (tp + fp)
recall = float(tp) / (tp + fn)
f1 = float(2 * precision * recall) / (precision + recall)

print('Precision: {} - Recall: {} - F1: {}'.format(precision, recall, f1))

In [ ]:
for idx, similarity in enumerate(similarity_data):
    reform = {(outerKey, innerKey): values for outerKey, innerDict in similarity.items() for innerKey, values in innerDict.items()}
    if idx == 0:
        tmp_df = pd.DataFrame(reform).T
    else:
        tmp_df = pd.concat([tmp_df, pd.DataFrame(reform).T])

In [ ]:
tmp_df

In [ ]:
tmp_df.describe()

In [11]:
tmp_df.to_csv(os.path.join(target_folder, 'comparison_sample_{}.csv'.format(threshold)))